In [25]:
#récupération de l'ap de bigml

from bigml.api import BigML
from pandas import read_csv

In [26]:
#récupération des identifiants du projet de bigml

%env BIGML_USERNAME=damienthiberge
%env BIGML_API_KEY=d92377e535638e44239dc49238540477263e1bff

env: BIGML_USERNAME=damienthiberge
env: BIGML_API_KEY=d92377e535638e44239dc49238540477263e1bff


In [27]:
#charge de l'api bigml pour pouvoir faire appel à ces fonctions

api = BigML()

In [28]:
#récupération du fichier csv bigml à traiter

train_full = api.create_source('../../handson-ml/mn_train.csv')

In [29]:
#test des sources

api.ok(train_full)

True

In [30]:
#création dataset depuis le fichier csv importer précédement

train_full_dataset = api.create_dataset(train_full)

In [31]:
#split du train full

train_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (80%)",
                     "sample_rate": 0.8, "seed": "Full train"})
validation_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (20%)",
                     "sample_rate": 0.8, "seed": "Full train",
                     "out_of_bag": True})

In [32]:
#creation d'un ensemble à partir de train 

ensemble = api.create_ensemble([train_dataset],{"objective_field":"SeriousDlqin2yrs"})

In [33]:
#batch prédiction

batch_prediction = api.create_batch_prediction(ensemble, validation_dataset, {"output_fields":['Id', 'SeriousDlqin2yrs']})

In [34]:
#test prediction

api.ok(batch_prediction)

True

In [35]:
#telechargement de la prédiction

api.download_batch_prediction(batch_prediction,
    filename='../../handson-ml/predictions_error.csv')

'../../handson-ml/predictions_error.csv'

In [36]:
error = read_csv('../../handson-ml/predictions_error.csv', index_col=0)

In [37]:
myList = list(error.columns)
myList[1] = 'Prediction'
error.columns = myList

In [38]:
error.to_csv("predictions_error.csv")

In [39]:
error.insert( 2, 'Error', '')

In [40]:
def errorPred(row):
    if row['SeriousDlqin2yrs'] == 0 and row['Prediction'] == 0:
        row['Error'] = 'TN'
    if row['SeriousDlqin2yrs'] == 0 and row['Prediction'] == 1:
        row['Error'] = 'FP'
    if row['SeriousDlqin2yrs'] == 1 and row['Prediction'] == 0:
        row['Error'] = 'FN'
    if row['SeriousDlqin2yrs'] == 1 and row['Prediction'] == 1:
        row['Error'] = 'TP'
    return row


In [41]:
for df in [error]:
    df['Error'] = df[['Error', 'SeriousDlqin2yrs', 'Prediction']].apply(errorPred,axis=1)
    
error.to_csv("predictions_error.csv")
error

,SeriousDlqin2yrs,Prediction,Error
Id,,,
6,0,0,TN
7,0,0,TN
13,0,0,TN
14,1,1,TP
21,0,0,TN
...,...,...,...
149970,0,0,TN
149975,0,0,TN
149984,0,0,TN


In [42]:
resultat = error['Error'].value_counts()
resultat

TN    27769
FN     1641
TP      348
FP      242
Name: Error, dtype: int64

In [45]:
precision = (27769+348)/30000
precision

0.9372333333333334